# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import gmaps.datasets

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
#convert ..WeatherPy/cityweather.csv into df
cityweather = "WeatherPy/cityweather.csv"
cityweather_df = pd.read_csv(cityweather)

cityweather_df

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hasaki,35.7333,140.8333,59.29,62,100,11.86,JP,1635377839
1,1,Broome,42.2506,-75.8330,50.38,85,75,3.00,US,1635377839
2,2,Punta Arenas,-53.1500,-70.9167,51.91,50,0,13.80,CL,1635377713
3,3,Faanui,-16.4833,-151.7500,79.74,78,67,6.11,PF,1635377840
4,4,Valdivia,-39.8142,-73.2459,59.09,97,2,3.06,CL,1635377545
...,...,...,...,...,...,...,...,...,...,...
545,545,Ambunti,-4.2365,142.8448,82.29,87,64,0.96,PG,1635377993
546,546,Venice of the Philippines,4.6628,119.3928,83.53,71,56,5.35,PH,1635377993
547,547,Chicama,-7.8447,-79.1469,61.34,81,32,8.55,PE,1635377994
548,548,Kilindoni,-7.9139,39.6668,77.77,81,40,6.96,TZ,1635377994


In [10]:
#dropping unnamed column
cityweather_df.drop(['Unnamed: 0'], axis=1, inplace=True)
cityweather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hasaki,35.7333,140.8333,59.29,62,100,11.86,JP,1635377839
1,Broome,42.2506,-75.8330,50.38,85,75,3.00,US,1635377839
2,Punta Arenas,-53.1500,-70.9167,51.91,50,0,13.80,CL,1635377713
3,Faanui,-16.4833,-151.7500,79.74,78,67,6.11,PF,1635377840
4,Valdivia,-39.8142,-73.2459,59.09,97,2,3.06,CL,1635377545
...,...,...,...,...,...,...,...,...,...
545,Ambunti,-4.2365,142.8448,82.29,87,64,0.96,PG,1635377993
546,Venice of the Philippines,4.6628,119.3928,83.53,71,56,5.35,PH,1635377993
547,Chicama,-7.8447,-79.1469,61.34,81,32,8.55,PE,1635377994
548,Kilindoni,-7.9139,39.6668,77.77,81,40,6.96,TZ,1635377994


In [17]:
#verifing data types for heatmap. humidity was int lat and long are floats
cityweather_df["Humidity"].astype(float)

#checking for NAN - none 

cityweather_df["Humidity"].isnull().values.any()

False

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#config for google heatmap
gmaps.config(api_key = g_key)



### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
